In [65]:
import json
import pdb
import codecs
import pdb
import pandas as pd

def get_t2_data():
    tweets = []
    files = ['racism.json', 'neither.json', 'sexism.json']
    for file in files:
        with codecs.open('/home/sjonnal3/Hate_Speech_Detection/Applied_Machine_Learning/tweet_data/' + file, 'r', encoding='utf-8') as f:
            data = f.readlines()
        for line in data:
            tweet_full = json.loads(line)
            tweets.append({
                'id': tweet_full['id'],
                'text': tweet_full['text'].lower(),
                'label': tweet_full['Annotation'],
                'name': tweet_full['user']['name'].split()[0]
                })

    #pdb.set_trace()
    return tweets



tweets = get_t2_data()

In [66]:
temp_data = pd.DataFrame(tweets)

temp_data['label'] = temp_data['label'].replace(["sexism","racism"],0)
temp_data['label'] = temp_data['label'].replace(["none"],1)

temp_data['label'].value_counts()

temp_data.columns = ['id','tweet','class','name']


In [67]:
temp_data.to_csv('/home/sjonnal3/Hate_Speech_Detection/Applied_Machine_Learning/chase-master/data/ml/public/dt/racism_sexism_2classes_only.csv', sep=',', encoding="utf-8")

In [68]:
raw_data = pd.read_csv('/home/sjonnal3/Hate_Speech_Detection/Applied_Machine_Learning/chase-master/data/ml/public/dt/racism_sexism_2classes_only.csv')

In [69]:
raw_data[raw_data['tweet'].isna()]

,Unnamed: 0,id,tweet,class,name
14992,#yesallmen,NaN,NaN,NaN,NaN


In [71]:
temp_data['tweet'].isna().sum()

0

In [28]:
help(temp_data.to_csv)

Help on method to_csv in module pandas.core.generic:

to_csv(path_or_buf: 'FilePath | WriteBuffer[bytes] | WriteBuffer[str] | None' = None, sep: 'str' = ',', na_rep: 'str' = '', float_format: 'str | Callable | None' = None, columns: 'Sequence[Hashable] | None' = None, header: 'bool_t | list[str]' = True, index: 'bool_t' = True, index_label: 'IndexLabel | None' = None, mode: 'str' = 'w', encoding: 'str | None' = None, compression: 'CompressionOptions' = 'infer', quoting: 'int | None' = None, quotechar: 'str' = '"', lineterminator: 'str | None' = None, chunksize: 'int | None' = None, date_format: 'str | None' = None, doublequote: 'bool_t' = True, escapechar: 'str | None' = None, decimal: 'str' = '.', errors: 'str' = 'strict', storage_options: 'StorageOptions' = None) -> 'str | None' method of pandas.core.frame.DataFrame instance
    Write object to a comma-separated values (csv) file.
    
    Parameters
    ----------
    path_or_buf : str, path object, file-like object, or None, defaul

In [61]:
raw_data = pd.read_csv('/home/sjonnal3/Hate_Speech_Detection/Applied_Machine_Learning/chase-master/data/ml/public/dt/racism_sexism_2classes_only.csv', sep=',', encoding="utf-8")

print(raw_data['class'].value_counts())

from nltk.corpus import stopwords

tweets = raw_data['tweet']
eng_stop_words = stopwords.words("english")

tweet_without_stopwords = tweets.apply(lambda x: ' '.join([word for word in x.split() if word not in (eng_stop_words)]))

1    11501
0     5406
Name: class, dtype: int64


In [49]:
import re

import nltk
from nltk import PorterStemmer, WordNetLemmatizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS

sentiment_analyzer = VS()
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words("english")
other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)



################################# Modifed the code from Char to Word level from Pinkesh paper ################################################

import sys
import re
from string import punctuation
# from preprocess_twitter import tokenize as tokenizer_g
from gensim.parsing.preprocessing import STOPWORDS

FLAGS = re.MULTILINE | re.DOTALL

def hashtag(text):
    text = text.group()
    hashtag_body = text[1:]
    if hashtag_body.isupper():
        result = u"<hashtag> {} <allcaps>".format(hashtag_body)
    else:
        result = " ".join(["<hashtag>"] + re.split("(?=[A-Z])", hashtag_body, flags=FLAGS))
    return result

def allcaps(text):
    text = text.group()
    return text.lower() + " <allcaps>"


def tokenizer_g(text):
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    text = re_sub(r"/"," / ")
    text = re_sub(r"@\w+", "<user>")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(r"<3","<heart>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(r"#\S+", hashtag)
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat>")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")

    ## -- I just don't understand why the Ruby script adds <allcaps> to everything so I limited the selection.
    # text = re_sub(r"([^a-z0-9()<>'`\-]){2,}", allcaps)
    text = re_sub(r"([A-Z]){2,}", allcaps)

    return text.lower()

def glove_tokenize(text):
    text = tokenizer_g(text)
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split()
    # words = [word for word in words if word not in STOPWORDS]
    return words

################################################################################################################


# stem_or_lemma: 0 - apply porter's stemming; 1: apply lemmatization; 2: neither
# -set to 0 to reproduce Davidson. However, note that because a different stemmer is used, results could be
# sightly different
# -set to 2 will do 'basic_tokenize' as in Davidson
def tokenize(tweet, stem_or_lemma=0):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and normalizes tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    if stem_or_lemma==0:
        tokens = [stemmer.stem(t) for t in tweet.split()]
    elif stem_or_lemma==1:
        tokens=[lemmatizer.lemmatize(t) for t in tweet.split()]
    else:
        tokens = [t for t in tweet.split()] #this is basic_tokenize in TD's original code
    return tokens


# tweets should have been preprocessed to the clean/right format before passing to this method
def get_pos_tags(tweets):
    """Takes a list of strings (tweets) and
    returns a list of strings of (POS tags).
    """
    tweet_tags = []
    for t in tweets:
        tokens = tokenize(t, 2)
        tags = nltk.pos_tag(tokens)
        tag_list = [x[1] for x in tags]
        #for i in range(0, len(tokens)):
        tag_str = " ".join(tag_list)
        tweet_tags.append(tag_str)
    return tweet_tags


In [53]:
import re
import enchant
import splitter

d = enchant.Dict('en_UK')
dus = enchant.Dict('en_US')
space_pattern = '\s+'
giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
mention_regex = '@[\w\-]+'
emoji_regex = '&#[0-9]{4,6};'

#This is the original preprocess method from Davidson
def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    parsed_text = re.sub('RT','', parsed_text) #Some RTs have !!!!! in front of them
    parsed_text = re.sub(emoji_regex,'',parsed_text) #remove emojis from the text
    parsed_text = re.sub('…','',parsed_text) #Remove the special ending character is truncated
    #parsed_text = re.sub('#[\w\-]+', '',parsed_text)
    #parsed_text = parsed_text.code("utf-8", errors='ignore')
    return parsed_text


def preprocess_clean(text_string, remove_hashtags=True, remove_special_chars=True):
    # Clean a string down to just text
    text_string=preprocess(text_string)

    parsed_text = preprocess(text_string)
    parsed_text = parsed_text.lower()
    parsed_text = re.sub('\'', '', parsed_text)
    parsed_text = re.sub('|', '', parsed_text)
    parsed_text = re.sub(':', '', parsed_text)
    parsed_text = re.sub(',', '', parsed_text)
    parsed_text = re.sub(';', '.', parsed_text)
    parsed_text = re.sub('&amp', '', parsed_text)

    if remove_hashtags:
        parsed_text = re.sub('#[\w\-]+', '',parsed_text)
    if remove_special_chars:
        #parsed_text = re.sub('(\!|\?)+','.',parsed_text) #find one or more of special char in a row, replace with one '.'
        parsed_text = re.sub('(\!|\?)+','',parsed_text)
    return parsed_text

def strip_hashtags(text):
    text = preprocess_clean(text,False,True)
    hashtags = re.findall('#[\w\-]+', text)
    for tag in hashtags:
        cleantag = tag[1:]
        if d.check(cleantag) or dus.check(cleantag):
            text = re.sub(tag,cleantag,text)
            pass
        else:
            hashtagSplit = ""
            for word in splitter.split(cleantag.lower(),'en_US'):
                hashtagSplit = hashtagSplit + word + " "
            text = re.sub(tag,hashtagSplit,text)
    #print(text)
    return text



In [56]:
from sklearn.feature_extraction.text import CountVectorizer
import functools


word_vectorizer = CountVectorizer(
        # vectorizer = sklearn.feature_extraction.text.CountVectorizer(
        # tokenizer=functools.partial(nlp.tokenize, stem_or_lemma=normalize_option),
        tokenizer = functools.partial(glove_tokenize),
        preprocessor=strip_hashtags,
        ngram_range=(1, 1),
        # stop_words=nlp.stopwords,  # We do better when we keep stopwords
        # stop_words = None,
        decode_error='replace',
        max_features=50000
        # min_df=2,
        # max_df=0.5
    )

counts = word_vectorizer.fit_transform(tweet_without_stopwords).toarray()

vocab = {v: i for i, v in enumerate(word_vectorizer.get_feature_names())}



/home/sjonnal3/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [58]:
tweet_without_stopwords

0        0 drasko said impressed girls... 1 drasko cook...
1        0 drasko said impressed girls... 1 drasko cook...
2        0 drasko said impressed girls... 1 drasko cook...
3        0 drasko said impressed girls... 1 drasko cook...
4        0 drasko said impressed girls... 1 drasko cook...
                               ...                        
16902    0 drasko said impressed girls... 1 drasko cook...
16903    0 drasko said impressed girls... 1 drasko cook...
16904    0 drasko said impressed girls... 1 drasko cook...
16905    0 drasko said impressed girls... 1 drasko cook...
16906    0 drasko said impressed girls... 1 drasko cook...
Name: tweet, Length: 16907, dtype: object

In [59]:
tweets = raw_data['tweet']

eng_stop_words = stopwords.words("english")

tweet_without_stopwords = tweets.apply(lambda x: ' '.join([word for word in x.split() if word not in (eng_stop_words)]))

In [63]:
raw_data

,Unnamed: 0,id,tweet,class,name
0,0,0 572342978255048705\n1 57234149...,0 so drasko just said he was impressed ...,0,0 thefoxbandit\n1 patricia\n...
1,1,0 572342978255048705\n1 57234149...,0 so drasko just said he was impressed ...,0,0 thefoxbandit\n1 patricia\n...
2,2,0 572342978255048705\n1 57234149...,0 so drasko just said he was impressed ...,0,0 thefoxbandit\n1 patricia\n...
3,3,0 572342978255048705\n1 57234149...,0 so drasko just said he was impressed ...,0,0 thefoxbandit\n1 patricia\n...
4,4,0 572342978255048705\n1 57234149...,0 so drasko just said he was impressed ...,0,0 thefoxbandit\n1 patricia\n...
...,...,...,...,...,...
16902,16902,0 572342978255048705\n1 57234149...,0 so drasko just said he was impressed ...,0,0 thefoxbandit\n1 patricia\n...
16903,16903,0 572342978255048705\n1 57234149...,0 so drasko just said he was impressed ...,0,0 thefoxbandit\n1 patricia\n...
16904,16904,0 572342978255048705\n1 57234149...,0 so drasko just said he was impressed ...,0,0 thefoxbandit\n1 patricia\n...
16905,16905,0 572342978255048705\n1 57234149...,0 so drasko just said he was impressed ...,0,0 thefoxbandit\n1 patricia\n...
